In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler


In [25]:
dataPath = '../../preprocess_train_dataset\correlation.csv'
dataframe = pd.read_csv(dataPath)

Feature Scaling

In [26]:
# Exclude the output column from feature scaling
columns_to_scale = [col for col in dataframe.columns if col != "Transported"]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
dataframe[columns_to_scale] = scaler.fit_transform(dataframe[columns_to_scale])



In [27]:
# Display the modified dataset after feature scaling
dataframe

,Earth,Europa,CryoSleep,Cabin_deck_B,Cabin_deck_C,Cabin_side_P,55 Cancri e,RoomService,Spa,VRDeck,Total_spending,Transported
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,False
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007608,0.024500,0.001823,0.020452,True
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.003001,0.299670,0.002030,0.288521,False
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.148563,0.007997,0.143830,False
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021149,0.025214,0.000083,0.030317,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.073322,0.003066,0.237197,False
8689,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,False
8690,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000045,0.000000,0.052047,True
8691,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.015753,0.134049,0.128852,False


Parameter Tuning

In [28]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning,module="sklearn")

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


train_x=dataframe.iloc[:, :-1].values
train_y=dataframe.iloc[:, -1].values



# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 10],
    'metric': ['euclidean', 'manhattan','minkowski']
}

# Initialize the K-Nearest Neighbors classifier
knn = KNeighborsClassifier()

# Perform grid search to find the best parameters
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the accuracy of the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy (Best Model): {:.2f}%".format(accuracy * 100))

#  Print the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)


Accuracy (Best Model): 77.80%
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 10}


prediction

In [29]:
test_dataPath = '../../preprocess_test_dataset/encoded_testData_01.csv'
df_test = pd.read_csv(test_dataPath)

print(df_test.shape)
print(df_test.columns)

(4277, 35)
Index(['PassengerId', 'HomePlanet', 'Earth', 'Europa', 'Mars', 'CryoSleep',
       'Cabin_deck', 'Cabin_deck_A', 'Cabin_deck_B', 'Cabin_deck_C',
       'Cabin_deck_D', 'Cabin_deck_E', 'Cabin_deck_F', 'Cabin_deck_G',
       'Cabin_deck_T', 'Cabin_num', 'Cabin_side', 'Cabin_side_P',
       'Cabin_side_S', 'Destination', '55 Cancri e', 'PSO J318.5-22',
       'TRAPPIST-1e', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Total_spending', 'First_Name', 'Last_Name',
       'First_Name_le', 'Last_Name_le'],
      dtype='object')


In [31]:

X_dataframe = pd.DataFrame(df_test)

dataframe.iloc[:, :-1]

# Select only the columns present in the training dataset
X_private_selected = X_dataframe[dataframe.iloc[:, :-1].columns]


# Make predictions on the test dataset
preds = grid_search.predict(X_private_selected)

# drop categorical data, remain the one hot encoded ones 
df_test_PassengerId = df_test['PassengerId']
output_path = '../../output_prediction/'

submission = pd.DataFrame({'PassengerId': df_test_PassengerId, 'Transported': preds})
submission.to_csv(output_path+'KNN.csv', index=False, index_label=False)


C:\Users\865u5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
